In [ ]:
#pip intall all needed libraries
pip install lxml
pip install tensorflow
pip install transformers
pip install awswrangler
pip install wordcloud
pip install nltk

# import nltk (natural language tool kit), a popular python package for text mining
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')

from os import path
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
# import nltk (natural language tool kit), a popular python package for text mining
import nltk
# stopwords, FreqDist, word_tokenize
from nltk import word_tokenize
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#stopwords = stopwords.words('english')
import matplotlib.pyplot as plt
# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Get data from s3
import boto3
import pandas as pd
import io

s3 = boto3.resource('s3')
bucket = s3.Bucket('bucket_name')
prefix_objs = bucket.objects.filter(Prefix="data_name")

prefix_df = []

for obj in prefix_objs:
    key = obj.key
    body = obj.get()['Body'].read()
    temp = pd.read_parquet(io.BytesIO(body))        
    prefix_df.append(temp)
df = pd.concat(prefix_df)
df.head()

#set the desired column to list
data = df.title.tolist()
data

#Clean data for simple word count
import lxml 
from html import unescape
import re
from bs4 import BeautifulSoup   # python package to handle HTML

def freq_analysis(texts):
    tokens = str(texts)
    #lowecases
    tokens = tokens.lower()
    # remove HTML related characters (&amp’,’&quot’,etc.)
    #soup = BeautifulSoup(unescape(tokens), 'lxml')
    # remove urls
    tokens = re.sub(r'http\S+', '', tokens)
    # Remove useless numbers and alphanumerical words
    tokens = re.sub("[^a-zA-Z0-9]", " ", tokens)
    #tokenization or word split
    tokens = word_tokenize(tokens)
    # Filter non-alphanumeric characters from tokens
    tokens = (word for word in tokens if word.isalpha())
    #additional stopwrods
    more_stopwords = set(('cant', 'aint', 'today'))
    #extra_stoplist = set(stopwords.words('english')) | more_stopwords
    #tokens = (word for word in tokens if word not in extra_stoplist)
    #remove short words
    tokens = (word for word in tokens if len(word) >= 3)
    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(tokens)
    
    #k refers to keys (or tokens); v refers to values (or counts)
    return  fdist

# identifying first 10 words
fdist = freq_analysis(data)
for k,v in list(fdist.items())[:10]:
    print(k,v)

def clean_data(texts):
    tokens = str(texts)
    #lowecases
    tokens = tokens.lower()
    # remove HTML related characters (&amp’,’&quot’,etc.)
    #soup = BeautifulSoup(unescape(tokens), 'lxml')
    # remove urls
    tokens = re.sub(r'http\S+', '', tokens)
    # Remove useless numbers and alphanumerical words
    tokens = re.sub("[^a-zA-Z0-9]", " ", tokens)
    #tokenization or word split
    tokens = word_tokenize(tokens)
    # Filter non-alphanumeric characters from tokens
    tokens = (word for word in tokens if word.isalpha())
    #additional stopwrods
    more_stopwords = set(('cant', 'aint', 'today'))
    #extra_stoplist = set(stopwords.words('english')) | more_stopwords
    #tokens = (word for word in tokens if word not in extra_stoplist)
    #remove short words
    tokens = (word for word in tokens if len(word) >= 3)
    
    #k refers to keys (or tokens); v refers to values (or counts)
    return tokens

data = clean_data(data)

#Performing Wordcloud

#mask = np.array(Image.open("data/butterfly.png"))

stopwords = set(STOPWORDS)
stopwords.add("said")
stopwords.add("the")
stopwords.add("you")
stopwords.add("yours")
stopwords.add("can")
stopwords.add("and")
stopwords.add("that")
stopwords.add("will")
stopwords.add("one")
stopwords.add("able")
stopwords.add("etc")
stopwords.add("has")
stopwords.add("done")
stopwords.add("see")
stopwords.add("new")
stopwords.add("first")

wordcloud = WordCloud(max_words=1000, stopwords=stopwords, margin=10,
               random_state=1).generate(' '.join(data))

#Plotting wordcloud
plt.figure(figsize=(16,16))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

#Set the desired column for sentitment analysis. Here we are using the title column
data = df.title.tolist()
data

# What is the overal sentiment? How many positive? how many negative? 

analyzer = SentimentIntensityAnalyzer()

finaldata = []

for i in data: 
    vs = analyzer.polarity_scores(i)
    finaldata.append([i, vs["compound"], vs['neg'], vs['neu'], vs['pos']])

len(finaldata)

# obtaining a dataframe for the text data.
df_2 = pd.DataFrame(finaldata)
df_2.columns = ['comments', 'compound', 'negative', 'neutral', 'positive']
df_2.head()

# Finding out the number of positive, negative and neutal words
positive_reviews = df_2.loc[df_2['compound'] > 0] # if compoundscore is > 0 then positive
negative_reviews = df_2.loc[df_2['compound'] < 0] # if compoundscore is < 0 then negative
neutral_reviews = df_2.loc[df_2['compound'] == 0] # if compoundscore is == 0 then neutral

print(len(positive_reviews))
print(len(negative_reviews))
print(len(neutral_reviews))

#Merging the sentiment dataframe with the original dataframe
df_2 = pd.concat([df, df_2.reindex(df.index)], axis=1)
df_2 = df_2[['place_id', 'comments', 'compound', 'negative', 'neutral', 'positive']]
df_2.rename(columns={'place_id': 'id'}, inplace=True)

# Changing all float data type to decimal since dynamoDB does not take in numbers
from decimal import Decimal, Context
ctx = Context(prec=5)
df_2.compound = df_2['compound'].apply(lambda x: ctx.create_decimal_from_float(x))
df_2.positive = df_2['positive'].apply(lambda x: ctx.create_decimal_from_float(x))
df_2.neutral = df_2['neutral'].apply(lambda x: ctx.create_decimal_from_float(x))
df_2.negative = df_2['negative'].apply(lambda x: ctx.create_decimal_from_float(x))
#df_2.compound = df_2.compound.apply(lambda x: Decimal(x))
df_2.id = df_2.id.astype(str)
df_2.head()

#Sentiment Analysis with HuggingFace
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
#data = ["I love you", "I hate you"]
results = sentiment_pipeline(data)

#Displaying first 10
results[:10]

#Put results to a dataframe
df_3 = pd.DataFrame(results)
df_3.head()

#Merging Sentiment scores with original dataframe
df_3 = pd.concat([df, df_3.reindex(df.index)], axis=1)
df_3 = df_3[['place_id', 'title', 'label', 'score']]
df_3.rename(columns={'place_id': 'id'}, inplace=True)
df_3.head()

#changing float to decimal
from decimal import Decimal, Context
ctx = Context(prec=5)
df_3.score = df_3['score'].apply(lambda x: ctx.create_decimal_from_float(x))
#df_2.compound = df_2.compound.apply(lambda x: Decimal(x))
df_3.id = df_3.id.astype(str)
df_3.head()

#Loading results to dynamoDB
import awswrangler as wr
wr.dynamodb.put_df(df=df_2, table_name="table_name")
wr.dynamodb.put_df(df=df_3, table_name="table_name")

#Delete objects in the s3 bucket
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket('bucket_name')
bucket_versioning = s3.BucketVersioning('bucket_name')
if bucket_versioning.status == 'Enabled':
    s3_bucket.object_versions.delete()
else:
    s3_bucket.objects.all().delete()


#shutting down notebook instance
import time
# using sleep() to hault the code execution
time.sleep(600)
print('Closing idle notebook')
client = boto3.client('sagemaker')
client.stop_notebook_instance(NotebookInstanceName= "notebook_instance_name")